In [ ]:
#松果更新庫存
import requests
import json
import time
from bs4 import BeautifulSoup
from googlesheet import delsheet
from googlesheet import getsheet
from googlesheet import writesheet
import xlrd


SCOPES = 'https://www.googleapis.com/auth/spreadsheets'

def getpcone(pid):
    url = "https://www.pcone.com.tw/product/info/"+pid
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    try:
        strnum = soup.select('script')[23].text.find("window._pc_p = ") #提取JSON資料
        gotjson = soup.select('script')[23].text[strnum+len("window._pc_p = "):-2]
        load = json.loads(gotjson)  #轉成python dict
    except:
        strnum = soup.select('script')[24].text.find("window._pc_p = ") #提取JSON資料
        gotjson = soup.select('script')[24].text[strnum+len("window._pc_p = "):-2]
        load = json.loads(gotjson)  #轉成python dict
    outObj = {"state":"0","prd":""}
    prdAry = []
    for i in range(len(load['volumes'])):
        prdAry.append([str(load['volumes'][i]['volume_id']),str(load['volumes'][i]['volume_remaining']),str(load['volumes'][i]['option'])])
    outObj["prd"] = prdAry
    return outObj

def getALLpcone():
    #prdidData = getsheet('商品ID!O:P') #獲取試算表所有資料
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(r"productsID.xlsx")
    table1 = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table1.nrows #取总行数

    
    wiAry=[]
    tiAry= []
    wrval = []
    wriso = [] #更新標題
    for i in range(rows_count):
        wriso.append([table1.cell_value(i,0),table1.cell_value(i,1),table1.cell_value(i,2)])

    for row in range(rows_count):
        wrval.append([""])
    
    for row in range(rows_count):
        print(str(row)+"/"+str(rows_count))
        try:
            if table1.cell_value(row,14):
                gotstock = getpcone(table1.cell_value(row,14))
                for i in range(len(gotstock["prd"])):
                    if gotstock["prd"][i][0] == table1.cell_value(row,15):
                        wrval[row] = [gotstock["prd"][i][1]]
                if wrval[row] == [""]:
                    wrval[row] = ["款式ID錯誤"]
        except:
            try:
                if table1.cell_value(row,14):
                    wrval[row] = ["錯誤或下架"]
            except:
                wrv                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                al[row] = [""]
    wrval[0][0] = "Pcone" + time.strftime("%m/%d", time.localtime())
    delsheet("test!Q:Q")
    writesheet("test!A1",wriso)
    writesheet("test!Q1",wrval)
    print("OK")

getALLpcone()

In [16]:
#偵測新上架
import requests
import json
import time
import tkinter as tk
from bs4 import BeautifulSoup
from googlesheet import delsheet
from googlesheet import getsheet
from googlesheet import writesheet
import xlrd


def clickbtn():
    try:
        output.delete('1.0',tk.END)
    except:
        print("start")
    pconedetect()

    
def pconedetect():

    url = "https://www.pcone.com.tw/api/merchant/products?items_per_page=1000&merchant_id=2945567&page=1"
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    data = json.loads(gotrq.text)  #轉成python dict
    pconeIDAry = []
    for i in range(len(data["products"])):
        pconeIDAry.append(data["products"][i]["display_id"])
    
    #GOTData = getsheet('商品ID!O:O') #獲取試算表所有資料
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(r"productsID.xlsx")
    table1 = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table1.nrows #取总行数
    
    GOTDataAry = table1.col_values(14)
    errorID = []
    #for i in range(len(GOTData)):
    #    try:
    #        GOTDataAry.append(GOTData[i][0])  #提取試算表資料
    #    except:
    #        GOTDataAry.append("") #空直例外
    for i in range(len(pconeIDAry)):#找網頁上架ID在不在 試算表中
        try:
            GOTDataAry.index(pconeIDAry[i])
        except: #找不到的話
            errorID.append(pconeIDAry[i])
    for i in range(len(errorID)):
        output.insert(1.0,str(errorID[i])+"\n")
    output.insert(1.0,"偵測完成\n")




gui=tk.Tk()
gui.title("查松果新上架")
output= tk.Text()
output.pack()
btn = tk.Button(gui, text ="執行", command = clickbtn)
btn.pack(anchor='center', expand='yes')
gui.mainloop()

In [12]:
#查ID
from bs4 import BeautifulSoup
import tkinter as tk
import requests
import json

def pconeID(pdid):

    url = "https://www.pcone.com.tw/product/info/"+pdid
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    try:
        strnum = soup.select('script')[23].text.find("window._pc_p = ") #提取JSON資料
        gotjson = soup.select('script')[23].text[strnum+len("window._pc_p = "):-2]
        load = json.loads(gotjson)  #轉成python dict
    except:
        strnum = soup.select('script')[24].text.find("window._pc_p = ") #提取JSON資料
        gotjson = soup.select('script')[24].text[strnum+len("window._pc_p = "):-2]
        load = json.loads(gotjson)  #轉成python dict
    output.insert(1.0,load['product_name'])
    for i in range(len(load['volumes'])):
        output.insert(1.0,str(load['volumes'][i]['volume_id'])+"   "+str(load['volumes'][i]['volume_remaining'])+" "+str(load['volumes'][i]['option'])+"\n")


def clickbtn():
    try:
        output.delete('1.0',tk.END)
    except:
        print("start")
    inputID = IDinput.get()
    pconeID(inputID)

    
gui=tk.Tk()
gui.title("查松果ID")
output= tk.Text()
output.pack()
IDinput = tk.Entry()
IDinput.pack()
btn = tk.Button(gui, text ="執行", command = clickbtn)
btn.pack(anchor='center', expand='yes')
gui.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "<ipython-input-12-488220f6d02c>", line 14, in pconeID
    strnum = soup.select('script')[23].text.find("window._pc_p = ") #提取JSON資料
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-12-488220f6d02c>", line 32, in clickbtn
    pconeID(inputID)
  File "<ipython-input-12-488220f6d02c>", line 18, in pconeID
    strnum = soup.select('script')[24].text.find("window._pc_p = ") #提取JSON資料
IndexError: list index out of range


In [ ]:
a = [['Pcone03/15'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], '', '', ['134'], ['15'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['20'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['1'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['88'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['39'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['92'], ['100'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], ['36'], '', ['錯誤或下架'], ['50'], ['50'], ['48'], '', '', '', '', ['14'], ['14'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['34'], '', '', '', '', '', '', '', '', '', ['0'], ['27'], ['0'], ['0'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], ['錯誤或下架'], '', '', '', ['20'], ['9'], ['7'], '', '', '', '', '', '', '', ['49'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['4'], ['10'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['50'], ['47'], ['20'], '', ['50'], '', ['50'], ['0'], ['50'], '', '', ['11'], '', '', '', '', '', '', '', '', '', '', '', ['0'], ['9'], ['0'], ['13'], ['錯誤或下架'], ['0'], ['15'], '', '', '', ['47'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['6'], ['0'], ['0'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['15'], ['10'], ['10'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], ['錯誤或下架'], '', '', '', ['錯誤或下架'], ['錯誤或下架'], ['錯誤或下架'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], '', ['20'], ['47'], '', '', '', '', '', '', '', '', ['15'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['254'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['20'], ['0'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['20'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['1'], ['29'], '', '', '', '', ['30'], ['0'], ['0'], '', '', ['94'], ['119'], ['94'], ['137'], ['141'], ['147'], ['0'], ['3'], '', '', '', '', '', '', ['2'], '', '', ['錯誤或下架'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['37'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], '', '', '', '', ['錯誤或下架'], '', ['錯誤或下架'], '', '', '', '', ['60'], '', '', '', '', ['87'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['9'], ['9'], ['8'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], '', '', '', '', '', '', ['35'], ['27'], '', '', '', '', '', '', '', '', '', '', '', '', '', ['15'], ['11'], '', '', '', '', '', ['14'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['26'], '', '', '', '', '', '', '', '', '', ['100'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['26'], '', '', '', '', '', '', '', '', '', '', ['38'], ['5'], ['35'], ['35'], ['39'], ['37'], ['38'], '', '', '', '', ['48'], '', '', '', '', '', '', '', '', '', '', '', '', '', ['5'], ['0'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['35'], '', '', '', '', '', '', '', ['20'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['款式ID錯誤'], '', '', '', '', '', '', '', '', '', '', '', ['錯誤或下架'], ['錯誤或下架'], ['4'], '', ['款式ID錯誤'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['50'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['50'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']